In [1]:
import src.midi_utils as midi_utils
import pygame
from pypianoroll import Multitrack, Track
import pypianoroll
import os
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
try:
    home_dir
except NameError:
    home_dir = os.getcwd()

os.chdir(home_dir + "/data/raw/bach")  # go to a folder relative to home dir


midi_filename = 'bwv110.7.mid'
csv_text = midi_utils.load_to_csv(midi_filename)
# get semitones to C major 
semitones,_ = midi_utils.get_semitones_to_C(csv_text)
print('Semitones to C major : ' +str(semitones))
bach_sample = pypianoroll.parse(midi_filename, beat_resolution=4)
# get pianoroll matrix (number of notes, all range)
bach_soprano_pianoroll = bach_sample.tracks[0].pianoroll
# create a track object 
track = Track(pianoroll=bach_soprano_pianoroll,name='new track')
transposed_track = pypianoroll.transpose(track,-semitones)
#print(transposed_track.pianoroll)
track_binarized = pypianoroll.binarize(track)
print(track_binarized.pianoroll[5])
print(np.argmax(transposed_track.pianoroll, axis=1))

# create a multitrack
new_multitrack = Multitrack(tracks=[transposed_track], beat_resolution=4)
#write to midi file 
pypianoroll.write(new_multitrack, home_dir + "/data/tests_midi/new_multitrack")


Semitones to C major : 2
[False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False  True
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False]
[ 0  0  0  0 69 69 71 71 72 72 72 72 71 71 71 71 69 69 69  0 69 69 71 71
 72 72 72 72 71 71 71 71 69 69 69 69 76 76 76 76 74 74 74 74 72 72 72 72
 71 71 71  0 71 71 71 71 72 72 72  0 72 72 72 72 

/Users/deperrois/Documents/BERN/ATML/myenv/lib/python3.7/site-packages/pretty_midi/pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  RuntimeWarning)


In [3]:
from src.dataset_utils import TrackDataset, get_dataset_representation_from_tracks
from torch.utils.data import DataLoader

from sklearn.preprocessing import StandardScaler

feature_scaler = StandardScaler()
label_scaler = StandardScaler()

all_pianorolls = []
# iterate over all midi files of folder
import glob
import os
import numpy as np

try:
    home_dir
except NameError:
    home_dir = os.getcwd()
    

In [4]:
# determine boundaries in all tracks
lower_notes = []
upper_notes = []
list_pianorolls = []
os.chdir(home_dir + "/data/raw/bach")  # go to a folder relative to home dir
for midi_file in glob.glob("*.mid"):
    # get a list of all soprano tracks
    ## load midi file
    csv_text = midi_utils.load_to_csv(midi_file)
    # get semitones to C major 
    semitones,_ = midi_utils.get_semitones_to_C(csv_text)
    # convert to multitrack object
    multitrack = pypianoroll.parse(midi_file, beat_resolution=4)
    # get the soprano track object and transpose it to C major 
    track = pypianoroll.transpose(multitrack.tracks[0], -semitones)
    # get pitch range 
    pitch_range = track.get_active_pitch_range()
    lower_notes.append(pitch_range[0])
    upper_notes.append(pitch_range[1])
    
    # get the flattened representation of pianoroll
    pianoroll_flattened = midi_utils.flatten_one_hot_pianoroll(track.pianoroll)
    # add it to the global list of all tracks
    list_pianorolls.append(pianoroll_flattened)

# convert into an array 
all_pianorolls = np.concatenate(list_pianorolls)

/Users/deperrois/Documents/BERN/ATML/myenv/lib/python3.7/site-packages/pretty_midi/pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  RuntimeWarning)


In [5]:
# get lower and upper bounds 
global_lower = min(lower_notes)
global_upper = max(upper_notes)
global_diff = global_upper - global_lower
n_notes = global_diff + 2 # we include the silence 
print('Global lower note : '+ str(global_lower))
print('Global upper note : '+ str(global_upper))
print('Number of notes : '+ str(n_notes))

Global lower note : 57
Global upper note : 83
Number of notes : 28


In [6]:
print(all_pianorolls[0:100])

[ 0  0  0  0 70 70 70  0 70 70 70 70 68 68 68 68 66 66 66 66 65 65 65 65
 65 65 65 65 63 63 63 63 63 63 63 63 70 70 70  0 70 70 70 70 72 72 72 72
 74 74 74 74 75 75 75 75 75 75 75 75 74 74 74 74 74 74 74 74 75 75 75 75
 78 78 78 78 77 77 77 77 77 77 75  0 75 75 75 75 75 75 75 75 75 75 75 75
 75 75 75 75]


In [7]:
# scale pianoroll to 0 
all_pianorolls_scaled = midi_utils.scale_pianoroll(all_pianorolls, global_lower)
print(all_pianorolls_scaled)

[ 0  0  0 ... 10 10 10]


## Define a recurrent-network model 

In [8]:
# Check if GPU is available
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')

No GPU available, training on CPU; consider making n_epochs very small.


Define a LSTM network as for character-to-character text generation

In [9]:
# Declaring the model
class CharRNN(nn.Module):
    
    def __init__(self, n_notes, n_hidden=256, n_layers=2,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        self.n_notes = n_notes 
        #define the LSTM
        self.lstm = nn.LSTM(self.n_notes, n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        #define a dropout layer
        self.dropout = nn.Dropout(drop_prob)
        
        #define the final, fully-connected output layer
        self.fc = nn.Linear(n_hidden, self.n_notes)
      
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
                
        #get the outputs and the new hidden state from the lstm
        r_output, hidden = self.lstm(x, hidden)
        
        #pass through a dropout layer
        out = self.dropout(r_output)
        
        # Stack up LSTM outputs using view
        out = out.contiguous().view(-1, self.n_hidden)
        
        #put x through the fully-connected layer
        out = self.fc(out)
        
        # return the final output and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden

## Training Code

Time for training! We declare a function, where we define an optimizer (Adam) and loss (cross entropy). We then create the training and validation data and initialize the hidden state of the RNN. 
We loop over the training set, each time encoding the data into one-hot vectors, performing forward and backpropagation, and updating the network parameters.

Every once a while, we generate some loss statistics (training loss and validation loss) to let us know if the model is training correctly.

In [10]:
# Declaring the train method
def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    ''' Training a network 
    
        Arguments
        ---------
        
        net: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss
    
    '''
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    if(train_on_gpu):
        net.cuda()
    
    counter = 0
    n_notes = net.n_notes 
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)
        
        for x, y in midi_utils.get_pianoroll_batches(data, batch_size, seq_length):
            counter += 1
            print("hey")
            
            # One-hot encode our data and make them Torch tensors
            x = midi_utils.one_hot_encode_batch(x, n_notes)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            net.zero_grad()
            
            # get the output from the model
            output, h = net(inputs, h)
            
            # calculate the loss and perform backprop
            loss = criterion(output, targets.view(batch_size*seq_length).long())
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in midi_utils.get_pianoroll_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode_batch(x, n_notes)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.view(batch_size*seq_length).long())
                
                    val_losses.append(val_loss.item())
                
                net.train() # reset to train mode after iterationg through validation data
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))

In [21]:
# Define and print the net
n_hidden=512
n_layers=2

net = CharRNN(n_notes, n_hidden, n_layers)
print(net)

# Declaring the hyperparameters
batch_size = 128
seq_length = 100
n_epochs = 20 # start smaller if you are just testing initial behavior

# train the model
train(net, all_pianorolls_scaled, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=50)

CharRNN(
  (lstm): LSTM(28, 512, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5)
  (fc): Linear(in_features=512, out_features=28, bias=True)
)
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey


KeyboardInterrupt: 

In [22]:
# Defining a method to generate the next character
def predict(net, note, h=None):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''
        
        # tensor inputs
        #x = np.array([[net.char2int[char]]])
        x = np.array([[note]])
        x = midi_utils.one_hot_encode_batch(x, net.n_notes)
        inputs = torch.from_numpy(x)
        
        if(train_on_gpu):
            inputs = inputs.cuda()
        
        # detach hidden state from history
        h = tuple([each.data for each in h])
        # get the output of the model
        out, h = net(inputs, h)

        # get the character probabilities
        p = F.softmax(out, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
        
        top_note = np.arange(net.n_notes)
        # select the likely next character with some element of randomness
        p = p.numpy().squeeze()
        pianoroll = np.random.choice(top_note, p=p/p.sum())
        
        # return the encoded value of the predicted char and the hidden state
        return pianoroll, h

In [23]:
# Declaring a method to generate new text
def sample(net, size, prime=[1,1,1,1]):
        
    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    net.eval() # eval mode
    
    # First off, run through the prime characters
    notes = [no for no in prime]
    h = net.init_hidden(1)
    for no in prime:
        note, h = predict(net, no, h)

    notes.append(note)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        note, h = predict(net, notes[-1], h)
        notes.append(note)

    return np.array(notes)

In [24]:
# Generating new melody
notes = sample(net, 100, prime=[1,1,1,1])
print(notes)
normal_pianoroll = midi_utils.unscale_pianoroll(notes, global_lower) # go back to the track range
print(normal_pianoroll)
one_hot_pianoroll = midi_utils.one_hot_encode(normal_pianoroll, 128)*90
new_track = Track(pianoroll=one_hot_pianoroll, name='new track')
# create a multitrack
new_multitrack = Multitrack(tracks=[new_track], beat_resolution=4)
#write to midi file 
pypianoroll.write(new_multitrack, home_dir + "/results/RNN_track")
modified_midi_filename = home_dir + "/results/RNN_track.mid"



[ 1  1  1  1  1 17 23  6  9 20  9  9 16 11 14  3 19  8 15 13 20 22  4 10
 17 15  9 18 15 23 12  0 16 11  6 10 10 16  4  6 15 15 14 11 16 14 14 12
  8  9 14 25 19 15 16 16 17 18 15 15 19  8 17  9 20 20 16 16  8 18 15 21
 12 13 18 15  0 11 18 14 10 24 13 18 14 17 15 14 16 13 16  8 15 11 21 16
 18  8 13 20 15 18 15 10 10]
[57 57 57 57 57 73 79 62 65 76 65 65 72 67 70 59 75 64 71 69 76 78 60 66
 73 71 65 74 71 79 68  0 72 67 62 66 66 72 60 62 71 71 70 67 72 70 70 68
 64 65 70 81 75 71 72 72 73 74 71 71 75 64 73 65 76 76 72 72 64 74 71 77
 68 69 74 71  0 67 74 70 66 80 69 74 70 73 71 70 72 69 72 64 71 67 77 72
 74 64 69 76 71 74 71 66 66]


In [25]:
pygame.init()
pygame.mixer.music.load(modified_midi_filename)
pygame.mixer.music.play()

In [18]:
pygame.mixer.music.stop()